# Copyright

<PRE>
Jelen iTorch notebook a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott "Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült. 
A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning
Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A notebook bármely részének újra felhasználása, publikálása csak a szerző írásos beleegyezése esetén megegengedett.

2016 (c) Tóth Bálint Pál (toth.b kukac tmit pont bme pont hu)
</PRE>

# Bevezető
Ebben a notebookban továbblépünk a korábbi XOR-os példákon és a sinus függvényt fogjuk modellezni. Fogunk találkozhatunk pár klasszikus jelenséggel, amellyekkel összetettebb adatok esetén is találkozhatunk. Először a korábbi forráskódot tisztázzuk le és írjuk át egy kicsit a célnak megfelelően. Pár dolgot az egyszerűség miatt kiveszünk belőle. Először definiáljunk három függvényt:

In [1]:
require "torch"
require "nn"
require "cutorch"
require "cunn"
require "optim"

mean=0
var=0

-- tanító adatok generálása
function genDataSin(from, to, resolution, standardscale, random)
    random=random or false
    if (standardscale==nil) then standardscale=true end
    
    local dat=torch.Tensor()
    if (random==false) then
        -- egymás utáni értékek generálálsa
        dat = torch.Tensor(resolution):range(from,to):apply(math.rad)
    else
        -- véletlenszerű értékek generálása
        dat = torch.rand(resolution):mul(to):add(from):apply(math.rad)
    end
    local result=torch.cat(dat:clone(), dat:clone():apply(math.sin)*0.9,2)

    -- bemenet standardizálás
    if (standardscale==true) then
        print("Calculating mean and standard deviation....")
        mean=result[{{},1}]:mean()
        var=result[{{},1}]:std()
    end
    
    result[{{},1}]=(result[{{},1}]-mean)/var
    
    return result:clone()
end

-- modell létrehozása
function createModel(inputs, hiddens, outputs)
    local model = nn.Sequential();
    model:add(nn.Linear(inputs, hiddens[1]))
    model:add(nn.Tanh())

    if table.getn(hiddens)>1 then
        for i=1,table.getn(hiddens)-1 do 
            model:add(nn.Linear(hiddens[i], hiddens[i+1]))
            model:add(nn.Tanh())
        end
    end
    
    model:add(nn.Linear(hiddens[table.getn(hiddens)], outputs))
    model:add(nn.Tanh())

    return model
end

-- modell tanítása
function trainModel()
   params, gradParams = model:getParameters()
   local batchInputs = torch.Tensor(batchSize, data:size(2)-1):cuda()
   local batchLabels = torch.DoubleTensor(batchSize,1):cuda()
    
   for i = 1,epochs do
        local shuffle = torch.randperm(data:size(1))
        for k=1,data:size(1),batchSize do 
            no=1
            for j=k,math.min(k+batchSize-1,data:size(1)) do
            batchInputs[no] = data[{shuffle[j],{1,-2}}]
            batchLabels[no] = data[{shuffle[j],-1}]
            no=no+1
        end

        local function feval(params)
            gradParams:zero() 
            local outputs = model:forward(batchInputs) 
            local loss = criterion:forward(outputs, batchLabels) 
            local dloss_doutput = criterion:backward(outputs, batchLabels)
            model:backward(batchInputs, dloss_doutput)
            return loss,gradParams
        end
        _,loss = optim.sgd(feval, params, optimState)
        losses[i]=loss[1]
    end
    if i%100==0 then print(loss[1]) end
    end
end

Ezután létrehozzuk a modellt, az adatokat és tanítunk:

In [2]:
cuda=true
epochs=500
batchSize=200
optimState = {
      learningRate = 0.05,
      weightDecay = 1e-4,
      momentum = 0.9,
      nesterov = false,
      dampering = 0,
      learningRateDecay = 0
    }

data=genDataSin(1,1000,1000)
criterion = nn.MSECriterion()
model=createModel(data:size(2)-1,{40,40},1)

if cuda==true then
    model:cuda()
    criterion:cuda()
    data=data:cuda()
end

if (batchSize>data:size(1)) then error() end 
losses = torch.Tensor(epochs)

print("Training started with batch size: " .. batchSize)
trainModel()
print("Training done.")

Calculating mean and standard deviation....	
Training started with batch size: 200	


0.3507893383503	


0.10427219420671	


0.033716902136803	


0.034543719142675	


0.02620524354279	
Training done.	


A tanítás után készítsünk véletlenszerű adatokat és teszteljük ezzel az eljárást:

In [3]:
function evalModel(data)
    local result =  model:forward(data)
    return result
end

test_data=genDataSin(1,1000,1000,false,true):cuda()
test_data=test_data:sort(1)
pred=evalModel(test_data[{{},{1,-2}}])


Majd jelenítsük meg az eredményt.

In [4]:
Plot = require 'itorch.Plot'

function plotResults(test_data,pred) 
    local plot=Plot()
    plot:line(data[{{},1}]:clone():mul(var):add(mean), data[{{},-1}],'blue' ,'Original')
    plot:line(test_data[{{},1}]:clone():mul(var):add(mean), pred:reshape(pred:size(1)),'red' ,'Predicted')
    plot:xaxis('Time (ms)')
    plot:legend(true):title('Modeling sin wave')
    plot:draw() --or plot:html()
end
plotResults(test_data,pred)

Nézzük meg a hiba csökkenését a tanulás során:

In [5]:
Plot = require "itorch.Plot"
 
local x = torch.linspace(0,epochs,epochs)
local plot = Plot()
plot:line(x, losses,'red' ,'Training loss')
plot:legend(true):title('Training XOR with simple model')
plot:draw()

Következőnek próbáljunk meg egy kicsit nagyobb tartományban jósolni, amelyik tartományból még nem jósoltunk:

In [6]:
test_data=genDataSin(500,1500,1000,false,true):cuda()
test_data=test_data:sort(1)
pred=evalModel(test_data[{{},{1,-2}}])
plotResults(test_data,pred)

Ez így nem lesz jó! Mit is csinálhatnánk? Próbáljuk meg a kimenetet "visszakötni" és ezáltal egy egyszerű rekurrens hálót szimulálni:

In [7]:
-- tanító adatok generálása
function genDataSinFeedback(from, to, resolution, feedback, standardscale)
    if (standardscale==nil) then standardscale=true end
    local dat = torch.Tensor(resolution):range(from,to):apply(math.rad)
    local result=dat:clone()
    for i=1,feedback do 
        result=torch.cat(result:clone(), torch.cat(torch.Tensor(i):zero(),dat[{{i+1,-1}}]):clone():apply(math.sin)*0.9,2)
    end
    
    result=torch.cat(result:clone(), dat:clone():apply(math.sin)*0.9,2)
    -- bemenet standardizálás
    if (standardscale==true) then
        print("Calculating mean and standard deviation....")
        mean=result[{{},1}]:mean()
        var=result[{{},1}]:std()
    end
    
    result[{{},1}]=(result[{{},1}]-mean)/var
    return result:clone()
end
data=genDataSinFeedback(1,1000,1000,1)

Calculating mean and standard deviation....	


És újra tanítjuk a hálót:

In [8]:
epochs=100
optimState = {
      learningRate = 0.05,
      weightDecay = 1e-4,
      momentum = 0.9,
      nesterov = false,
      dampering = 0,
      learningRateDecay = 0
    }

criterion = nn.MSECriterion()
model=createModel(data:size(2)-1,{40,40},1)

if cuda==true then
    model:cuda()
    criterion:cuda()
    data=data:cuda()
end

if (batchSize>data:size(1)) then error() end 
losses = torch.Tensor(epochs)

print("Training started with batch size: " .. batchSize)
trainModel()
print("Training done.")

Training started with batch size: 200	


In [9]:
test_data=genDataSinFeedback(500,3000,1000,1,false):cuda()
--test_data=test_data:sort(1)
pred=evalModel(test_data[{{},{1,-2}}])
plotResults(test_data,pred)

Végül jelenítsük meg a hiba csökkenését a visszakötött kimenetű tanítás során (láthatjuk, hogy sokkal gyorsabban tanul a hálózat):

In [10]:
Plot = require "itorch.Plot"
 
local x = torch.linspace(0,epochs,epochs)
local plot = Plot()
plot:line(x, losses,'red' ,'Training loss')
plot:legend(true):title('Training XOR with simple model')
plot:draw()

# További próbálkozási lehetőségek
Még számos jelenséget lehet vizsgálni a sin példáján:
- neuronok és rétegek számának a módosítása
- SGD algoritmus paramétereinek a módosítása
- L1 és L2 regularizáció megvalósítása (lásd XOR példa)
- ne az előre elkészített kimenetet kössük vissza, hanem a tényleges kimenetet generálás során
- kössünk vissza több korábbi kimenetet
- ne egyessével léptessük a visszakötést (tehát ne a -1, -2, -3, ... időpillanatokat, hanem pl. -5,-10,-15, ... időpillanatokat kössük vissza)
- próbáljuk meg többdimenziós kimenettel
- a bemenethez is vehetünk további változókat